In [79]:

import pandas as pd
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=2,
              batch_size=128, callbacks=callbacks_list)




In [80]:
notes = []
log = []

annees = [2002]
notes = []
for x in annees:
    notesAnnees = pd.read_pickle(r'../data/results/notes_' + str(x))
    notes += notesAnnees
notes = notes[0:10000]
n_vocab = len(set(notes))

In [ ]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab)

print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/2
78/78 [==============================] - 208s 2s/step - loss: 6.4438
Epoch 2/2
75/78 [===========================>..] - ETA: 8s - loss: 6.0010 

In [ ]:
print('creating model for prediction')
print('be sure to have the correct path for weights...')
model = create_network(network_input, n_vocab)
model.load_weights('weights-improvement-08-4.7016-bigger.hdf5')

In [ ]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

In [ ]:
prediction_output

# 

In [78]:
# 

creating model for prediction


ValueError: Cannot assign to variable dense_71/kernel:0 due to variable shape (256, 284) and value shape (256, 179) are incompatible

In [76]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

generating notes


In [77]:
prediction_output

['E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3',
 'E-4.F#4.B-3']

# 